In [1]:
import cv2
import numpy as np
import mediapipe as mp
from utils import draw_landmarks, calculate_landmarks
import time
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
import torch
import mediapipe as mp
from datasets import GestureDataset
import pandas as pd
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning import LightningModule, Trainer
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm as tqdm

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
class GestureRecognizer(pl.LightningModule):
    def __init__(self, lr=0.001):
        super().__init__()
        mp_hands_sol = mp.solutions.hands
        self.mp_hands = mp_hands_sol.Hands(
            max_num_hands=1,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5,
        )
        self.lr = lr
        self.l1 = nn.Linear(64, 128)
        self.l2 = nn.Linear(128, 128)
        self.l3 = nn.Linear(128, 6)

    def forward(self, x):
        x = x.float()
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

    def loss_fn(self, out, target):
        loss = nn.CrossEntropyLoss()(out, target)
        return loss

    def training_step(self, batch, batch_idx):
        x, y = batch[0]
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [3]:
ds = GestureDataset('poses.csv')


In [4]:
epochs = 30
lr = 0.001
batch_size = 64

model = GestureRecognizer(lr)

In [5]:
train_loader = DataLoader(ds, batch_size=batch_size)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
trainer = Trainer(
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
    max_epochs=epochs,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(model=model, train_dataloaders=[train_loader])


  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 8.3 K 
1 | l2   | Linear | 16.5 K
2 | l3   | Linear | 774   
--------------------------------
25.6 K    Trainable params
0         Non-trainable params
25.6 K    Total params
0.102     Total estimated model params size (MB)
/home/pavle/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


`Trainer.fit` stopped: `max_epochs=30` reached.
2022-09-17 10:31:10.978254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-17 10:31:11.091409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pavle/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-09-17 10:31:11.091423: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-17 10:31:11.107830: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register 

2022-09-17 10:31:11.583470: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pavle/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-09-17 10:31:11.583518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pavle/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-09-17 10:31:11.583529: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [8]:
m = torch.jit.script(model)
torch.jit.save(m, 'gesture_recognizer.pt')

/home/pavle/.local/lib/python3.10/site-packages/torch/jit/_recursive.py:621: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getattr(mod, name, None)
/home/pavle/.local/lib/python3.10/site-packages/torch/jit/_recursive.py:708: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getattr(nn_module, name, None)
/home/pavle/.local/lib/python3.10/site-packages/torch/jit/_recursive.py:508: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getattr(nn_module, name, None)
/home/pavle/.local/lib/python3.10/site-packages/torch/jit/_recursive.py:591: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  item = getat